<a href="https://colab.research.google.com/github/gusteivos/IJCA-final-challenge/blob/main/IJCA-final-challenge-agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:

!pip install pandas google-adk google-generativeai -q


In [27]:


!mkdir -p /content/medallion/silver

!wget -P /content/medallion/silver https://raw.githubusercontent.com/gusteivos/IJCA-final-challenge/refs/heads/main/medallion/silver/datatran.csv -q
!wget -P /content/medallion/silver https://raw.githubusercontent.com/gusteivos/IJCA-final-challenge/refs/heads/main/medallion/silver/acidentes.csv -q



In [28]:

import re


In [29]:

import google.generativeai as genai
from google.adk.runners import Runner
from google.genai.types import Content, Part


In [30]:

from google.colab import userdata
import os


agent_grupo4_api_key = userdata.get("IJCA-final-challenge-agent-grupo4")
genai.configure(api_key = agent_grupo4_api_key)

os.environ["GOOGLE_API_KEY"] = agent_grupo4_api_key


In [31]:

# https://google.github.io/adk-docs/tools-custom/function-tools/#example
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.query.html

from typing import Optional
import pandas as pd



def get_datatran_csv(
    start_date: Optional[str] = None,
    start_time: Optional[str] = None,
    end_date: Optional[str] = None,
    end_time: Optional[str] = None,
    head: Optional[int] = None,
    query: Optional[str] = None,
    count: Optional[str] = None,
    sum: Optional[str] = None,
    mean: Optional[str] = None,
):
    """
    Carrega e filtra o arquivo 'medallion/silver/datatran.csv', permitindo filtros
    por data, hora, consultas com Pandas e operações de agregação simples.

    O arquivo lido possui as seguintes colunas:

        data,dia_semana,hora,uf,br,km,municipio,causa_acidente,tipo_acidente,
        classificacao_acidente,fase_dia,sentido_via,condicao_metereologica,
        tipo_pista,tracado_via,uso_solo,pessoas,mortos,feridos_leves,
        feridos_graves,ilesos,ignorados,feridos,veiculos,latitude,longitude,
        regional,delegacia,uop, id(adicionado por comodidade)

    Caminho fixo do dataset:
        medallion/silver/datatran.csv

    Args:
        start_date (str, opcional): Filtra linhas com data >= start_date (YYYY-MM-DD).
        start_time (str, opcional): Filtra linhas com hora >= start_time (HH:MM:SS).
        end_date (str, opcional): Filtra linhas com data <= end_date (YYYY-MM-DD).
        end_time (str, opcional): Filtra linhas com hora <= end_time (HH:MM:SS).
        head (int, opcional): Limita o número de linhas retornadas.
        query (str, opcional): Expressão Pandas `.query()` aplicada ao DataFrame.
        count (str, opcional): Nome da coluna para aplicar count().
        sum (str, opcional): Nome da coluna para aplicar sum().
        mean (str, opcional): Nome da coluna para aplicar mean().

    Returns:
        str: CSV contendo dados filtrados ou o resultado da agregação.
    """

    df = pd.read_csv("medallion/silver/datatran.csv")
    df.insert(0, "id", range(1, len(df) + 1))

    if start_date is not None:
        df = df[df["data"] >= start_date]

    if start_time is not None:
        df = df[df["hora"] >= start_time]

    if end_date is not None:
        df = df[df["data"] <= end_date]

    if end_time is not None:
        df = df[df["hora"] <= end_time]

    if query is not None:
        df = df.query(query)

    if head is not None:
        df = df.head(head)

    if count is not None:
        return pd.DataFrame({"count": [df[count].count()]}).to_csv(index=False)

    if sum is not None:
        return pd.DataFrame({"sum": [df[sum].sum()]}).to_csv(index=False)

    if mean is not None:
        return pd.DataFrame({"mean": [df[mean].mean()]}).to_csv(index=False)

    return df.to_csv(index=False)


In [32]:

from google.adk.agents import Agent
from google.adk.tools import FunctionTool


def create_agent_grupo4():
  return Agent(
      name="IJCAfinalchallengeagentgrupo4",
      model="gemini-2.5-flash",
      instruction="""
Você é um agente especializado em análise de dados de acidentes de trânsito utilizando os conjuntos de dados disponibilizados pela Polícia Rodoviária Federal (PRF) do Brasil.
Você possui acesso à ferramenta get_datatran_csv, que permite consultar, filtrar e recuperar dados estruturados sobre acidentes (incluindo datas, horários, localizações, gravidade, veículos e outras variáveis relevantes).
Seu objetivo é responder perguntas com base nesses dados, sempre que possível utilizando a ferramenta para obter informações atualizadas diretamente do dataset.
Quando necessário, você também pode realizar análises estatísticas, sumarizações, comparações, explicações e visualizações conceituais.
Caso uma pergunta exija dados reais do conjunto, sempre chame a tool get_datatran_csv com os parâmetros adequados.
Caso a pergunta seja conceitual ou explicativa, você pode responder diretamente.
      """,
      tools=[get_datatran_csv]
  )

agent_grupo4 = create_agent_grupo4()


In [33]:

import asyncio
from google.adk.sessions import InMemorySessionService, Session
from IPython.display import display, Markdown


# --- A Helper Function to Run Our Agents ---
# We'll use this function throughout the notebook to make running queries easy.

async def run_agent_query(agent: Agent, query: str, session: Session, user_id: str, is_router: bool = False):
    """Initializes a runner and executes a query for a given agent and session."""
    print(f"\n🚀 Running query for agent: '{agent.name}' in session: '{session.id}'...")

    runner = Runner(
        agent=agent,
        session_service=session_service,
        app_name=agent.name
    )

    final_response = ""
    try:
        async for event in runner.run_async(
            user_id=user_id,
            session_id=session.id,
            new_message=Content(parts=[Part(text=query)], role="user")
        ):
            if not is_router:
                # Let's see what the agent is thinking!
                print(f"EVENT: {event}")
            if event.is_final_response():
                final_response = event.content.parts[0].text
    except Exception as e:
        final_response = f"An error occurred: {e}"

    if not is_router:
     print("\n" + "-"*50)
     print("✅ Final Response:")
     display(Markdown(final_response))
     print("-"*50 + "\n")

    return final_response

# --- Initialize our Session Service ---
# This one service will manage all the different sessions in our notebook.
session_service = InMemorySessionService()


my_user_id = "IJCA-final-challenge-agent-test-user"


In [34]:

async def run_agent_grupo4():

  my_session = await session_service.create_session(
    app_name=agent_grupo4.name,
    user_id=my_user_id
    )

  query = "Quantos acidentes ocorreram no Brasil no dia 2025-01-15 entre 14:00 e 18:00?" # @param {type: "string", placeholder: "Converse com o agente do grupo 4."}

  await run_agent_query(agent_grupo4, query, my_session, my_user_id)


In [35]:

await run_agent_grupo4()



🚀 Running query for agent: 'IJCAfinalchallengeagentgrupo4' in session: 'e936d0a6-ad70-4650-865a-31a1bdfa8cb7'...
EVENT: model_version='gemini-2.5-flash' content=Content(
  parts=[
    Part(
      function_call=FunctionCall(
        args={
          'count': 'id',
          'end_date': '2025-01-15',
          'end_time': '18:00:00',
          'start_date': '2025-01-15',
          'start_time': '14:00:00'
        },
        id='adk-1f8acf8b-bc30-4868-8a68-0912dabd170b',
        name='get_datatran_csv'
      ),
      thought_signature=b"\n\x9d\x04\x01\xd1\xed\x8ao\x92\xb2J\xb8DO\xdci$i\xfd%\xfe\x14\xd0\xca'\x99\x94`D\x11\xc8H\x7f\n\x00\xce\x03\x9d\xcc\xadI{ww\xf5\x9e_N\x07\x90\xfd\x95I;\x9dn\xf6/\xa6\x9a?\xda\x9d\xa9\xd6d\xee\xdd\xbbv,_V\xf6\xd7\x10\xd9H\x81\x97\xd4\xfd\xe0\x88 \x88>\xe8\xf2\xad\xfc\xf3\x0e\xd1!\xd2\x8e...'
    ),
  ],
  role='model'
) grounding_metadata=None partial=None turn_complete=None finish_reason=<FinishReason.STOP: 'STOP'> error_code=None error_message=None inte

No dia 15 de janeiro de 2025, entre 14:00 e 18:00, ocorreram 51 acidentes no Brasil.

--------------------------------------------------

